In [4]:
import re
import os
from textwrap import dedent
from pprint import pprint
from typing import List
from datetime import datetime

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableConfig, chain

from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableLambda

# Chat template message classes
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# 환경변수 로드
load_dotenv()

# ----- 도구 정의 -----
@tool
def tavily_search_func(query: str) -> str:
    """최신 트렌드, 뉴스, 실시간 정보를 Tavily API로 검색합니다."""
    tavily = TavilySearchResults(max_results=2)
    docs = tavily.invoke(query)
    if not docs:
        return "관련 정보를 찾을 수 없습니다."
    return "\n---\n".join(
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
    )

def wiki_search_and_summarize(input_data: dict):
    from langchain_community.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    docs = loader.load()
    return [
        f'<Document source="{doc.metadata.get("source", "Wikipedia")}"/>\n{doc.page_content}\n</Document>'
        for doc in docs
    ]

class WikiSummarySchema(BaseModel):
    query: str = Field(..., description="The query to search for in Wikipedia")

summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | ChatPromptTemplate.from_template("Summarize the following text:\n\n{context}\n\nSummary:")
    | ChatOpenAI(
        base_url="https://api.groq.com/openai/v1",
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        temperature=0.7
    )
)
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        위키백과에서 정보를 찾아 요약합니다.
        배경지식이 필요할 때 유용합니다.
    """),
    args_schema=WikiSummarySchema
)

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """카페 메뉴 벡터 DB에서 메뉴 정보를 검색합니다."""
    embeddings = OllamaEmbeddings(model="bge-m3:latest")
    db = FAISS.load_local("./db/cafe_db", embeddings, allow_dangerous_deserialization=True)
    results = db.similarity_search(query, k=3)
    return results if results else [Document(page_content="관련 카페 메뉴 정보를 찾을 수 없습니다.")]

tools = [tavily_search_func, wiki_summary, db_search_cafe_func]

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

llm_with_tools = llm.bind_tools(tools=tools)

# ----- Few-shot 예제 정의 -----
examples = [
    HumanMessage(content="아메리카노의 가격과 특징, 그리고 커피의 역사에 대해 알려주세요.", name="example_user"),
    AIMessage(content="카페 메뉴를 검색하고, 위키피디아에서 커피 역사 정보를 찾아보겠습니다.", name="example_assistant"),
    AIMessage(content="", name="example_assistant", tool_calls=[{"name": "db_search_cafe_func", "args": {"query": "아메리카노"}, "id": "1"}]),
    AIMessage(content="아메리카노: 가격 ₩4,500, 에스프레소와 뜨거운 물로 만든 클래식 블랙 커피, 원두 본연의 맛을 느낄 수 있음", name="example_assistant"),
    AIMessage(content="아메리카노 정보를 찾았습니다. 이제 커피의 역사를 위키피디아에서 검색해보겠습니다.", name="example_assistant"),
    AIMessage(content="", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "커피 역사"}, "id": "2"}]),
    AIMessage(content="커피는 에티오피아에서 기원하여 아랍을 거쳐 전 세계로 전파된 음료입니다. 15세기 예멘에서 처음 재배되기 시작했으며, 17세기 유럽으로 전해져 커피하우스 문화가 발달했습니다. 산업혁명과 함께 대량 생산이 가능해지면서 현재와 같은 커피 문화가 형성되었습니다.", name="example_assistant"),
    AIMessage(content="아메리카노(₩4,500)는 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 커피는 에티오피아에서 기원하여 아랍을 거쳐 전 세계로 전파되었으며, 15세기 예멘에서 처음 재배되기 시작했습니다. 17세기 유럽으로 전해져 커피하우스 문화가 발달했고, 산업혁명과 함께 대량 생산이 가능해지면서 현재와 같은 커피 문화가 형성되었습니다.", name="example_assistant"),
]

# ----- Few-shot 프롬프트 템플릿 -----
today = datetime.today().strftime("%Y-%m-%d")

FEWSHOT_PROMPT = ChatPromptTemplate.from_messages([
    ("system", dedent(f"""
    오늘 날짜는 {today}입니다.
    당신은 카페 메뉴 정보와 일반적인 음식/음료 지식을 제공하는 AI입니다.

    도구 사용 가이드라인:
    - db_search_cafe_func: 카페 메뉴 정보 (가격, 재료, 설명)
    - wiki_summary: 일반 지식 (역사, 제조법, 문화적 배경)
    - tavily_search_func: 최신 정보 (트렌드, 뉴스, 실시간 정보)

    사용 원칙:
    1. 카페 메뉴 관련 질문 → 반드시 메뉴 DB 먼저 검색
    2. 역사/문화/일반 지식 → 위키피디아 활용
    3. 최신 트렌드/뉴스 → 웹 검색 활용
    4. 복합 질문 → 여러 도구 순차 사용
    5. 정보 출처를 명확히 구분하여 답변
    """)),
    # Few-shot 대화 예시 (위 examples 사용)
    *[(msg.type, msg.content) for msg in examples],
    MessagesPlaceholder("history"),
    ("human", "{user_input}")
])

# ----- 고급 체인 구현 -----
@chain
def cafe_fewshot_chain(user_input: str, config: RunnableConfig):
    # 1. 프롬프트 생성
    history = config.get("history", [])
    prompt = FEWSHOT_PROMPT.format_prompt(user_input=user_input, history=history)

    # 2. LLM 도구 선택 및 호출
    ai_msg = llm_with_tools.invoke(prompt.to_messages(), config=config)
    tool_results = []

    # 3. Tool call 해석 및 실행
    for call in getattr(ai_msg, "tool_calls", []):
        name = call["name"]
        value = None
        try:
            if name == "tavily_search_func":
                value = tavily_search_func.invoke(call, config=config)
            elif name == "wiki_summary":
                value = wiki_summary.invoke(call, config=config)
            elif name == "db_search_cafe_func":
                value = db_search_cafe_func.invoke(call, config=config)
        except Exception as e:
            value = f"{name} 도구 실행 중 오류: {e}"
        tool_results.append(str(getattr(value, "content", value)))

    # 4. 도구 결과 합성 및 최종 답변 프롬프트 구성
    results_text = "\n\n".join(tool_results)
    final_prompt = ChatPromptTemplate.from_messages([
        ("system", "아래 도구 결과를 참고하여, 정보 출처를 명확히 구분해 사용자 친화적으로 답변하세요."),
        ("human", "{user_input}"),
        ("ai", ai_msg.content or "도구를 사용해 정보를 찾았습니다."),
        ("human", "도구 결과:\n{tool_results}")
    ])
    return (final_prompt | llm).invoke({
        "user_input": user_input,
        "tool_results": results_text
    }, config=config)

# ----- 테스트 코드 -----
if __name__ == "__main__":
    config = {"history": []}
    query = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."
    response = cafe_fewshot_chain.invoke(query, config=config)
    print("질문:", query)
    print("답변:", response.content)

질문: 카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요.
답변: ## 카페라떼와 어울리는 디저트

메뉴 DB에 따르면, 카페라떼와 어울리는 디저트는 다음과 같습니다.

*   **마카롱**: 마카롱은 라떼의 풍부한 맛과 잘 어울리는 프랑스 디저트입니다. 마카롱의 부드럽고 달콤한 맛이 라떼의 고소한 맛과 조화롭습니다.
*   **크로아상**: 크로아상은 라떼와 함께 먹기 좋은 프랑스 빵입니다. 크로아상의 버터 향이 라떼의 고소한 맛과 잘 어울립니다.
*   **스콘**: 스콘은 라떼와 함께 먹기 좋은 영국 디저트입니다. 스콘의 부드럽고 달콤한 맛이 라떼의 고소한 맛과 조화롭습니다.

## 라떼의 유래

위키피디아에 따르면, 라떼의 유래는 다음과 같습니다.

*   라떼는 이탈리아의 커피 음료로, 17세기부터 존재해 왔습니다.
*   라떼의 이름은 이탈리아어로 "우유"를 의미하는 "latte"에서 유래했습니다.
*   라떼는 에스프레소와 우유를 섞어 만든 음료로, 이탈리아에서 인기가 많습니다.

라떼는 다양한 종류가 있으며, 카페라떼, 마끼아또, 플랫 화이트 등이 있습니다. 라떼는 커피와 우유의 조합으로, 커피의 풍미와 우유의 부드러움을 동시에 즐길 수 있는 음료입니다.
